In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
import collections
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
###################
n_f=7
n_node=365
###################
#label - load fact
y_area=np.load('data/data_cablearea.npy');print('y_area.shape',y_area.shape)
###node ddata 
#location feature
node_loc=(pd.read_excel('data/raw_data/node_location.xlsx',header=None).values)
#node_loc=np.array([node_loc]*1201);print('node_loc',node_loc.shape)
nf=np.load('data/nf.npy')
disp=np.load('data/disp.npy')#(12000, 365, 7)
#x=np.concatenate((disp,node_loc),axis=-1);
x=disp.copy();print(x.shape)#(1201, 365, 10)
#############################
features=x.copy()
label_area=y_area.copy()
for i in range(7):
    print(np.min(features[:,:,i]),np.max(features[:,:,i]),np.mean(features[:,:,i]),np.std(features[:,:,i]))
for i in range(7): print(np.min(nf[:,i]),np.max(nf[:,i]),np.mean(nf[:,i]),np.std(nf[:,i]))
#split
np.random.seed(1)
indices = np.random.permutation(len(features));print(indices[0])
n_training=int(len(features)*0.6)+4#6:1:3#7:1:2
n_val=int(len(features)*0.2)-2
n_test=len(features)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
print(f_training.shape,f_val.shape,f_test.shape)
l_training, l_val, l_test = label_area[training_idx],label_area[val_idx], label_area[test_idx]
print(l_training.shape,l_val.shape,l_test.shape)

#multi task
def getting_multi(label_data,sf,mode):
    c_data=np.argmin(label_data,axis=1)
    a_data=np.min(label_data,axis=1)
    #c_data=c_data+1;c_data[a_data==1e-06]=0
    c_data=c_data+1
    c_data[a_data>0.999]=0;print('safe',np.sum(c_data==0))
    a_data=a_data[:,np.newaxis]
    
    bce_data = np.zeros((len(c_data), 121))
    bce_data[np.arange(len(c_data)),c_data] = 1

    if sf:
        np.savez_compressed('data/label_multitask/c_label_'+mode,c_data)
        np.savez_compressed('data/label_multitask/a_label_'+mode,a_data)
        np.savez_compressed('data/label_multitask_bce/c_label_'+mode,bce_data)
        np.savez_compressed('data/label_multitask_bce/a_label_'+mode,a_data)
    return c_data,a_data,bce_data

save_flag=True#False#
c_training,a_training,bce_training=getting_multi(l_training,sf=save_flag,mode='train')
c_val,a_val,bce_val=getting_multi(l_val,sf=save_flag,mode='val')
c_test,a_test,bce_test=getting_multi(l_test,sf=save_flag,mode='test')

#save
save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'features_train',f_training)
    np.savez_compressed(data_path+'features_val',f_val)
    np.savez_compressed(data_path+'features_test',f_test)
    data_path='data/label/'
    np.savez_compressed(data_path+'label_train',l_training)
    np.savez_compressed(data_path+'label_val',l_val)
    np.savez_compressed(data_path+'label_test',l_test)

y_area.shape (12121, 120)
(12121, 365, 7)
-1.0 1.0 -0.09185825200163956 0.3833312739758173
-1.0 1.0 -0.09065604719768403 0.3423182800463426
-1.0 1.0 -0.23030432059033284 0.3568375258278978
-1.0 1.0 0.04283109927425155 0.35493911239980064
-1.0 1.0 0.048609124054642384 0.3404480481062101
-1.0 1.0 0.02170862502513716 0.4052780665698899
-1.0 1.0 0.04228783399968983 0.33943828960502476
0.2891176130449867 0.29663907923229804 0.2962112152020598 0.0009263896923062869
0.6193100885613426 0.6332320162107397 0.6322039760916405 0.0020061946986215267
0.7661074082586379 0.7757350089209527 0.7744890372013288 0.001724427323117476
1.087003782773164 1.089265290561516 1.0884728652635887 0.0008346176073050315
1.1735987231245892 1.2030798845043311 1.2009738323241608 0.004431020782836264
1.4357914082242131 1.4468011227176711 1.4455269579585555 0.0018089297602514503
1.5672016048144435 1.590558445070064 1.5889022428325215 0.0033593363682782543
8203
(7276, 365, 7) (2422, 365, 7) (2423, 365, 7)
(7276, 120) (2422

In [1]:
import numpy as np
import pickle
import collections
from sklearn.preprocessing import MinMaxScaler
    
adj_mx = np.load('data/sensor_graph/adj_mx.npy')
for i in range(len(adj_mx)): adj_mx[i,i]=0
print(adj_mx.shape,np.sum(adj_mx))
edge_data=np.array([[i,j] for i,j in zip(np.where(adj_mx==1)[0],np.where(adj_mx==1)[1])])

#####distance 
edge_d=np.load('data/sensor_graph/dist_mx.npy')
for i in range(len(adj_mx)): edge_d[i,i]=0
edge_d=edge_d[edge_data[:,0],edge_data[:,1]]

#####virtual
edge_v=np.load('data/sensor_graph/virtual_mx.npy')
for i in range(len(adj_mx)): edge_v[i,i]=0
edge_v=edge_v[edge_data[:,0],edge_data[:,1]]

#####element type
#type 1
edge_t1=np.load('data/sensor_graph/type_c_mx.npy')
for i in range(len(adj_mx)): edge_t1[i,i]=0
edge_t1=edge_t1[edge_data[:,0],edge_data[:,1]]
#type 2
edge_t2=np.load('data/sensor_graph/type_t_mx.npy')
for i in range(len(adj_mx)): edge_t2[i,i]=0
edge_t2=edge_t2[edge_data[:,0],edge_data[:,1]]
#type 3
edge_t3=np.load('data/sensor_graph/type_g_mx.npy')
for i in range(len(adj_mx)): edge_t3[i,i]=0
edge_t3=edge_t3[edge_data[:,0],edge_data[:,1]]
#type 4
edge_t4=np.load('data/sensor_graph/type_cb_mx.npy')
for i in range(len(adj_mx)): edge_t4[i,i]=0
edge_t4=edge_t4[edge_data[:,0],edge_data[:,1]]

#concatenate
edge_f=np.stack([edge_d,edge_v,edge_t1,edge_t2,edge_t3,edge_t4],axis=1)#edge_f=np.stack([edge_d,edge_e,edge_a,edge_v],axis=1)
print(edge_data.shape,edge_f.shape)

#scale
sc= MinMaxScaler(feature_range=(-1,1)).fit(edge_f[:,:1])
edge_f[:,:1]=sc.transform(edge_f[:,:1])
    
for i in range(edge_f.shape[1]):
    print(i,np.min(edge_f[...,i]),np.max(edge_f[...,i]),np.mean(edge_f[...,i]),np.std(edge_f[...,i]))


save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'edge_features',edge_f)
    np.savez_compressed(data_path+'edge_data',edge_data)

(365, 365) 486.0
(486, 2) (486, 6)
0 -1.0 1.0 -0.65416276 0.56123763
1 0.0 1.0 0.012345679 0.11042311
2 0.0 1.0 0.24691358 0.43121606
3 0.0 1.0 0.09876543 0.29834682
4 0.0 1.0 0.39506173 0.48886397
5 0.0 1.0 0.24691358 0.43121606
